In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np
from IPython.display import HTML
import datetime
import pandas as pd
from re import I
import itertools
import matplotlib.animation as animation
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.interpolate import make_interp_spline

read_file = pd.read_excel('50 ml sevkiyatlar sade.xlsx')
read_file.to_csv ("Test.csv",
index = None,
header=True)
df = pd.DataFrame(pd.read_csv("Test.csv",parse_dates=True))

data_time = df.groupby(['Plan. Sevk. Tarih/Saat'])['Bağlı Satış Mik.'].sum()
dt = data_time[2:]
idx = pd.date_range('2023-01-02', '2023-12-31')
dt.index = pd.DatetimeIndex(dt.index)
dt = dt.reindex(idx, fill_value=0)
dt.head(),dt.shape

In [ ]:
# SCATTER PLOT #

fig, ax = plt.subplots(figsize=(40,10))

plt.style.use("seaborn-v0_8-dark")

for param in ['figure.facecolor', 'axes.facecolor', 'savefig.facecolor']:
    plt.rcParams[param] = '#%02x%02x%02x' % (0, 0, 0) # '#212946'   bluish dark grey
for param in ['text.color', 'axes.labelcolor', 'xtick.color', 'ytick.color']:
    plt.rcParams[param] = '0.9'  # very light grey

colors = [
    '#08F7FE',  # teal/cyan '#%02x%02x%02x' % (0, 128, 64)
    '#FE53BB',  # pink
    '#F5D300',  # yellow
    '#00ff41', # matrix green
    ]

ax.set_facecolor("black")

n_drops = 364

data = np.array([np.array([i for i in range(364)]),np.array(dt.values)]).reshape(364,-2)

rain_drops = np.zeros(n_drops, dtype=[#('position', float, (2,)),
                                      ('size',     float),
                                      ('growth',   float),
                                      #('color',    float, (4,))
                                      ])

rain_drops['growth'] = np.array(dt.values)/8000
rain_drops['size'] = 0
scat = ax.scatter(np.array([i for i in range(364)]), np.array(dt.values),
               s=rain_drops['size'],
               c="pink",
               alpha=0.98
                  )


def update(i):
    rain_drops['size'] += rain_drops['growth']
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    scat.set_sizes(rain_drops['size'])
    #kw = dict(prop="sizes", num=6, color=scat.cmap(0.3), fmt="{x}",func=lambda s: s)
    #ax.legend(*scat.legend_elements(**kw),labelspacing=4)


anima = animation.FuncAnimation(fig, update, frames=80, interval=40 ,repeat = False)
FFwriter = animation.FFMpegWriter(fps=30)
anima.save('scatter_sales.mp4', writer = FFwriter)
#HTML(anima.to_html5_video())

In [ ]:
# BAR PLOT #

fig, ax1 = plt.subplots(figsize=(40,10))

x = [i+1 for i in range(364)]
y = dt.values

ax1.set_facecolor("black")

data = np.column_stack([np.linspace(0, yi, 80) for yi in y])

ax1.set_ylim(0, max(y))


def animate(i):
    ax1.cla()
    ax1.set_ylim(0, max(y))
    rects = ax1.bar(x, data[i], color='c',align="center")
    ax1.bar_label(rects, padding=5,rotation=90,fontsize=8,color="grey")
    ax1.set_xticklabels([])
    ax1.set_yticklabels([])


anim1 = animation.FuncAnimation(fig, animate, frames=len(data), interval=40,repeat=False)

FFwriter = animation.FFMpegWriter(fps=30)
anim1.save('bar_sales.mp4', writer = FFwriter)
#HTML(anim1.to_html5_video())

In [ ]:
# LINE PLOT #

fig, ax = plt.subplots(figsize=(40,10))

x = dt.index
y = dt.values

ax.set_facecolor("black")
line, = ax.plot(x,y, label='Forecast', color='#00ff41')
plt.tick_params(labelsize=15)


def update(num):
    line.set_data(x[:num], y[:num])
    ax.set_xticklabels([])
    return line,

anim2 = animation.FuncAnimation(fig, update, frames=len(x),
                               interval=8,
                              repeat=False)
FFwriter = animation.FFMpegWriter(fps=140)
anim2.save('line_sales.mp4', writer = FFwriter)
#HTML(anim2.to_html5_video())